# Introdução

## Motivação

Utilizando o conhecimento obtido no curso até este ponto, já temos em mãos o ferramental necessário para construir diversas calculadoras e análises dentro do contexto de Renda Fixa do mercado financeiro. A construção de ferramentas mais complexas vai depender do aprofundamento e completo entendimento de conceitos que passamos de forma mais superficial aqui - até pela necessidade de conhecimentos matemáticos mais avançados em alguns casos. Porém, para muitas coisas no Mercado Financeiro, não precisamos levar à mão ferramentas complexas. A exemplo disto, vamos fazer um projeto muito interessante e que resulta em uma calculadora importante. Vamos Precificar uma NTN-F.
<br>

## O que é uma NTN-F?

A Nota do Tesouro Nacional - Série F (sigla NTN-F) é simplesmente o nome antigo do que hoje a plataforma do Tesouro Direto chama de "Tesouro Prefixado com Juros Semestrais". A mudança do nome é devido ao fato de a plataforma desejar tornar os investimentos mais acessíveis ao público de varejo do país e, desta forma, deixar o nome mais intuitivo para o investidor inexperiente.
<br>
Este título tem três características principais:
1. É um título prefixado, ou seja, sua taxa é fixada no momento da compra;
2. É um título que paga cupons semestrais, ou seja, ele possui uma <b>taxa de cupom</b> e um fluxo de caixa durante sua vida;
3. É um título emitido pelo tesouro Nacional, ou seja, não possui embutido em sua taxa nenhum <i>spread</i> de crédito. Em outras palavras, ele não possui nenhuma remuneração adicional pelo risco de calote - o risco deste título é o que chamamo de <b>risco soberano</b> - e usamos estes títulos soberanos para, posteriormente, calcular quanto de <b>prêmio de risco</b> as emissões corporativas estão pagando.

# Precificando o Título

Vamos fazer a precificação do título em passo-a-passo.
<br>
A primeira coisa que precisamos obter são as características do título. Para isto, podemos cotar a taxa em uma instituição, ou, a título de didática, podemos olhar na própria plataforma do tesouro Direto. Lá veremos a seguinte taxa para a data de consulta:
<br>
![Tela do tesouro Direto](tela-tesouro-direto.png)

Aqui podemos identificar as seguintes informações-chave:
1. A taxa ofertada é de 11,37%;
2. O título vence em 01/01/2033;
<br>

Além disso, sabemos que este produto (basta ler no descritivo do produto no próprio site do Tesouro Direto) que:
1. Ele paga cupons semestrais com uma taxa de cupom de 10% a.a.;
2. Este título vale, no vencimento R$ 1.000,00.
<br>

Com estas informações em mãos, podemos começar a implementar nossa precificação.

## Passo 1

O primeiro passo é importarmos as bibliotecas que vamos usar neste projeto, e inicializar as variáveis características do títulos que vamos precificar.

In [2]:
import numpy as np
import pandas as pd

from datetime import datetime, date
from dateutil.relativedelta import relativedelta

In [3]:
VF = 1000 
taxaTitulo = 0.1137       # 11,37% a.a
taxaCupom = 0.1000        # 10% a.a
periodicidadeCumpom = 2   # 2 vezes ao ano
vencimento = date(2033, 1, 1)

## Passo 2

O segundo passo é buscarmos, no site da ANBIMA, o calendário de feriados bancários, pois sabemos que a NTN-F remunera o investidor em <b>dias úteis</b>.

P.S.: Site da ANBIMA: https://www.anbima.com.br/feriados/

In [ ]:
def buscarCalendarioANBIMA() -> pd.DataFrame: 
  calendario = pd.read_excel('https://www.anbima.com.br/feriados/arqs/feriados_nacionais.xls')
  calendario.dropna(subset=['Dia da Semana', 'Feriado'], how='any', inplace=True)  
  calendario = calendario[['Data']] 
  calendario['Data'] = [x.date() for x in calendario['Data']]

  return calendario

In [5]:
calendarioFeriados = buscarCalendarioANBIMA()
calendarioFeriados.head(10)

,Data
0,2001-01-01
1,2001-02-26
2,2001-02-27
3,2001-04-13
4,2001-04-21
5,2001-05-01
6,2001-06-14
7,2001-09-07
8,2001-10-12
9,2001-11-02


## Passo 3

O terceiro passo é montarmos o calendário de datas relevantes para o título, ou seja, precisamos obter a data de cada evento na vida deste título - fazendo correções em caso de datas que caiam nos finais de semana ou feriados!

In [ ]:
def montandoDatasFluxo(vcto: date, calendario: pd.DataFrame) -> list:
  hoje = date.today()
  while hoje.weekday() == 5 or hoje.weekday() == 6 or hoje in calendario['Data'].values.tolist():
    hoje = hoje - relativedelta(days=1)

  datasRelevantes = []
  dataRelevante = vcto
  while dataRelevante >= hoje:
    dataAuxiliar = dataRelevante

    while dataAuxiliar.weekday() == 5 or dataAuxiliar.weekday() == 6 or dataAuxiliar in calendario['Data'].values.tolist():
      dataAuxiliar = dataAuxiliar + relativedelta(days=1)

    datasRelevantes.append(dataAuxiliar)

    dataRelevante = dataRelevante - relativedelta(months=6)
  
  return datasRelevantes

In [7]:
datasFluxos = montandoDatasFluxo(vcto=vencimento, calendario=calendarioFeriados)
datasFluxos[:3]

[datetime.date(2033, 1, 3),
 datetime.date(2032, 7, 1),
 datetime.date(2032, 1, 2)]

## Passo 4

O quarto passo é determinarmos qual o <b>Valor Futuro</b> que vai acontecer nestas datas de interesse que obtivemos.

In [8]:
def montandoValoresFuturos(valorFace: float, taxaCupom: float, datasInteresse: list, perCpm: int) -> list:
  valorCupom = round(valorFace * round((1+taxaCupom)**((12/perCpm)/12)-1, 8), 6)
  cupons = []
  for idx, i in enumerate(datasInteresse):
    if idx == 0:
      cupons.append(valorFace+valorCupom)
    else:
      cupons.append(valorCupom)
  
  return cupons

In [9]:
fluxoValoresFuturos = montandoValoresFuturos(valorFace=VF, taxaCupom=taxaCupom, datasInteresse=datasFluxos, perCpm=periodicidadeCumpom)
fluxoValoresFuturos[:3]

[1048.80885, 48.80885, 48.80885]

## Passo 5

O quinto passo é obtermos o <b>Valor Presente</b> dos fluxos de caixa que vão ocorrer. Como já temos as datas destes fluxos e o valor destes fluxos, precisamos criar a inteligência para trazer cada um a valor presente, dada a sua distância temporal da data de precificação (hoje).

Aproveitando que, neste ponto do projeto, já teremos todas as variáveis calculadas, podemos mandar a função já retornar o <i>dataframe</i> final pronto para visualizarmos!

In [ ]:
def montandoValoresPresentes(taxa: float, datas: list, valoresFuturos: list, calFeriados: list) -> pd.DataFrame:
  hoje = date.today()
  while hoje.weekday() == 5 or hoje.weekday() == 6 or hoje in calFeriados['Data'].values.tolist():
    hoje = hoje - relativedelta(days=1)

  numeroDUs = []
  valorCupom = []
  fatores = []
  for idx, i in enumerate(datas):
    dus = np.busday_count(hoje, i, holidays=[x.strftime('%Y-%m-%d') for x in calFeriados['Data'].values.tolist()])-1
    fator = round(1/ (1+taxa)**(dus/252), 9)
    valPres = round(valoresFuturos[idx] * fator, 9)

    numeroDUs.append(dus)
    fatores.append(fator)
    valorCupom.append(valPres)

  dfFluxos = pd.DataFrame()
  dfFluxos['Data'] = datas  
  dfFluxos['Dias Úteis'] = numeroDUs
  dfFluxos['Valor Futuro'] = valoresFuturos
  dfFluxos['Valor Presente'] = valorCupom
  dfFluxos['Fator'] = fatores
  dfFluxos.sort_values(by=['Data'], inplace=True, ignore_index=True)

  return dfFluxos

In [20]:
df = montandoValoresPresentes(taxa=taxaTitulo, datas=datasFluxos, valoresFuturos=fluxoValoresFuturos, calFeriados=calendarioFeriados)
df

,Data,Dias Úteis,Valor Futuro,Valor Presente,Fator
0,2025-07-01,73,48.80885,47.309748,0.969286
1,2026-01-02,203,48.80885,44.753208,0.916908
2,2026-07-01,325,48.80885,42.479796,0.870330
3,2027-01-04,452,48.80885,40.235807,0.824355
4,2027-07-01,575,48.80885,38.175556,0.782144
5,2028-01-03,703,48.80885,36.143490,0.740511
6,2028-07-03,827,48.80885,34.278133,0.702293
7,2029-01-02,951,48.80885,32.509046,0.666048
8,2029-07-02,1075,48.80885,30.831261,0.631674
9,2030-01-02,1200,48.80885,29.227574,0.598817


## Passo 6

O sexto passo é obtermos o Preço Unitário (na sigla PU) do título que estamos precificando. O preço de um título de Renda Fixa soberano não é nada mais e nada menos do que a soma de todos os fluxos de caixa que esperamos receber por ele. Faz sentido?
<br>
Você concorda que ninguém estaria disposto a pagar mais do que $X$ reais para receber $X$ reais em valor futuro. E se esse título valesse menos, todos os participantes informados de mercado iriam aplicar nele até que seu preço fosse corrigido (pela lei da oferta - maior a demanda pelos títulos e com oferta limitada, aumenta seu valor novamente). Desta forma, podemos dizer que o preço do título vai ser exatamente o somatório da coluna "Valor Presente".

In [21]:
PU = round(sum(df['Valor Presente']), 6)
print(f'O preço do titulo deve ser: R${round(PU, 2)}')

O preço do titulo deve ser: R$955.4


# Obtendo algumas métricas de sensibilidade

Agora que somos capazes de precificar uma NTN-F, podemos também calcular sua <i>duration</i> e sua convexidade para termos uma noção de <b>sensibilidade</b> do preço deste título - essa sensibilidade serve tanto para gestão de risco quanto para gestão do portfolio em si.

## <i>Duration</i>

In [22]:
duration = 0
for idx, i in enumerate(df['Valor Presente']):
  duration += i * df['Dias Úteis'].iloc[idx]

duration = duration / sum(df['Valor Presente'])
print(round(duration, 2))

1364.55


Lembrando a equação que define a <i>duration</i>, temos:
$$\Delta P = -D \cdot P \cdot \Delta y$$

Então podemos verificar, através da <i>duration</i>, <b>quanto irá variar o PU de um título dada uma variação em sua taxa</b>.

In [23]:
variacaoPU = -duration/252*PU*0.0001
print(f'A sensibilidade do titulo a uma variação de 0,01% na taxa é de: R${round(variacaoPU, 2)}')

A sensibilidade do titulo a uma variação de 0,01% na taxa é de: R$-0.52


## Convexidade

In [25]:
convexidade = 0
for idx, i in enumerate(df['Valor Presente']):
  convexidade += i * (df['Dias Úteis'].iloc[idx]/252)**2

convexidade = convexidade / sum(df['Valor Presente'])
print(round(convexidade, 2))

36.66


Lembrando que podemos expandir agora a equação do preço do título com uma segunda derivada da expansão de <i>Taylor</i>, ficando com:
$$\Delta P = -DP \Delta y + \frac{1}{2} C P (\Delta y)^2$$
<br>
Podemos, então, agregar mais este termo na equação de sensibilidade do título!

In [26]:
variacaoPU2 = -duration/252*PU*0.0001 + 0.5*convexidade*PU*(0.0001)**2
print(f'A sensibilidade do título a uma variação de 0,01% na taxa é de: R${round(variacaoPU2, 2)}')

A sensibilidade do título a uma variação de 0,01% na taxa é de: R$-0.52


In [28]:
print(abs(variacaoPU - variacaoPU2))

0.00017510945972809555
